<a id="TableOfContents"></a>
# Table of Contents:
<li><a href="#Q1">Question 1</li></a>
<li><a href="#Q2">Question 2</li></a>
<li><a href="#Q3">Question 3</li></a>
<li><a href="#Q4">Question 4</li></a>
<li><a href="#Q5">Question 5</li></a>
<li><a href="#testing">TESTING (evaluation.py)</li></a>

In [1]:
import pandas as pd
import sklearn.metrics
from sklearn.metrics import confusion_matrix
import evaluation

<a id="Q1"></a>
### 1. Create a new file named model_evaluation.py or model_evaluation.ipynb for these exercises.
<li><a href="#TableOfContents">Table of Contents</li></a>

COMPLETED

<a id="Q2"></a>
### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.
<li><a href="#TableOfContents">Table of Contents</li></a>

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |


##### 2a. In the context of this problem, what is a false positive?

- predicted dog, actual cat

##### 2b. In the context of this problem, what is a false negative?

- Predicted cat, actual dog

##### 2c. How would you describe this model?

It's rainin' doggos 'un kitties

<a id="Q3"></a>
### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.
<li><a href="#TableOfContents">Table of Contents</li></a>

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions <a href="https://ds.codeup.com/data/c3.csv">can be found here.</a>

Use the predictions dataset and pandas to help answer the following questions:

In [2]:
c3 = pd.read_csv('c3.csv')
c3.sample()

,actual,model1,model2,model3
140,Defect,No Defect,Defect,Defect


In [3]:
pd.crosstab(c3.actual, c3.model1)

model1,Defect,No Defect
actual,,
Defect,8,8
No Defect,2,182


In [4]:
confusion_matrix(c3.actual, c3.model1, labels=('No Defect', 'Defect'))

array([[182,   2],
       [  8,   8]])

In [5]:
confusion_matrix(c3.actual, c3.model1, labels=('No Defect', 'Defect'))[0, 1]

2

##### 3a. An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

Assuming 'No Defect' is desired outcome...

I want a model to predict the most defects...

I want to use specificity:

TN / (FP + TN)

- TN df[col][1, 1]
    - Predicted Defect
    - Actual Defect
- FP df[col][1, 0]
    - Predicted No Defect
    - Actual Defect

In [6]:
evaluation.specificity(c3, 'actual', 'No Defect', 'Defect')

actual: 100.00%

model1: 80.00%

model2: 10.00%

model3: 13.13%



In [7]:
for col in c3:
    matrix = confusion_matrix(c3.actual, c3[col], labels=('No Defect', 'Defect'))
    result = (matrix[1, 1] / (matrix[1, 0] + matrix[1, 1]))
    print(f'\033[32m{col}:\033[0m {result}\n')

actual: 1.0

model1: 0.5

model2: 0.5625

model3: 0.8125



model3 is the best model to predict defects

##### 3b. Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

Assuming 'No Defect' is the desired outcome...

I want the model with the best prediction to actual...

I want to use accuracy:

(TP + TN) / (TP + FP + FN + TN)

In [8]:
evaluation.accuracy(c3, 'actual')

actual: 100.00%

model1: 95.00%

model2: 56.00%

model3: 55.50%



In [9]:
for col in c3:
    ratio = (c3[col] == c3['actual']).mean()
    print(f'\033[32m{col}:\033[0m {ratio}\n')

actual: 1.0

model1: 0.95

model2: 0.56

model3: 0.555



model1 is the best to predict correct outcomes

Correction, if I'm looking for the model that predicts the correct outcome...

Assuming 'No Defect' is the desired outcome...

I want to use recall...

TP / TP + FN
- TP
    - Predicted 'No Defect'
    - Actual 'No Defect'
- FN
    - Predicted 'Defect'
    - Actual 'No Defect

In [10]:
evaluation.recall(c3, 'actual', 'No Defect', 'Defect')

actual: 100.00%

model1: 95.79%

model2: 93.64%

model3: 97.03%



In [11]:
for col in c3:
    matrix = confusion_matrix(c3.actual, c3[col], labels=('No Defect', 'Defect'))
    ratio = (matrix[0, 0] / (matrix[0, 0] + matrix[0, 1]))
    print(f'\033[32m{col}:\033[0m {ratio:.2%}')

actual: 100.00%
model1: 98.91%
model2: 55.98%
model3: 53.26%


<a id="Q4"></a>
### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).
<li><a href="#TableOfContents">Table of Contents</li></a>

At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

Several models have already been developed with the data, and <a href="https://ds.codeup.com/data/gives_you_paws.csv">you can find their results here.</a>

Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [12]:
paws = pd.read_csv('gives_you_paws.csv')
paws.sample()

,actual,model1,model2,model3,model4
4799,dog,dog,cat,cat,dog


In [13]:
# Baseline/Frequent = dog
paws.actual.value_counts()

dog    3254
cat    1746
Name: actual, dtype: int64

In [14]:
paws['baseline'] = 'dog'

##### 4a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [15]:
evaluation.accuracy(paws, 'actual')

actual: 100.00%

model1: 80.74%

model2: 63.04%

model3: 50.96%

model4: 74.26%

baseline: 65.08%



models 1 and 4 do better than the baseline

##### 4b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?

Assuming 'dog' is the desired outcome...

I want to predict 'dog'...

I want to use precision...

TP / (TP + FP)
- TP
    - Predicted dog
    - Actual dog
- FP
    - Predicted cat
    - Actual dog

In [16]:
evaluation.precision(paws, 'actual', 'dog', 'cat')

actual: 100.00%

model1: 80.33%

model2: 49.08%

model3: 50.86%

model4: 95.57%

baseline: 100.00%



In [17]:
for col in paws:
    matrix = confusion_matrix(paws.actual, paws[col], labels=('dog', 'cat'))
    ratio = (matrix[0, 0] / (matrix[0, 0] + matrix[0, 1]))
    print(f'\033[32m{col}:\033[0m {ratio:.2%}')

actual: 100.00%
model1: 80.33%
model2: 49.08%
model3: 50.86%
model4: 95.57%
baseline: 100.00%


model4 is the best at predicting 'dog'

##### 4c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?

Assuming 'cat' is the desired outcome...

I want the model that predicts 'cat' the best...

I want to use precision...

TP / (TP + FP)
- TP
    - Predicted cat
    - Actual cat
- FP
    - Predicted dog
    - Actual cat

In [18]:
evaluation.precision(paws, 'actual', 'cat', 'dog')

actual: 100.00%

model1: 81.50%

model2: 89.06%

model3: 51.15%

model4: 34.54%

baseline: 0.00%



model1 is the best model

<a id="Q5"></a>
### 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.
<li><a href="#TableOfContents">Table of Contents</li></a>

##### 5a. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html">sklearn.metrics.accuracy_score</a>

In [19]:
sklearn.metrics.accuracy_score(paws.actual, paws.model1)

0.8074

##### 5b. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html">sklearn.metrics.precision_score</a>

In [20]:
sklearn.metrics.precision_score(paws.actual, paws.model1, labels=['cat', 'dog'], pos_label='dog')

0.8900238338440586

##### 5c. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html">sklearn.metrics.recall_score</a>

In [21]:
sklearn.metrics.recall_score(c3.actual, c3.model1, labels=['No Defect', 'Defect'], pos_label='No Defect')

0.9891304347826086

##### 5d. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html">sklearn.metrics.classification_report</a>

In [22]:
sklearn.metrics.classification_report(paws.actual, paws.model1, labels=['dog', 'cat'])

'              precision    recall  f1-score   support\n\n         dog       0.89      0.80      0.84      3254\n         cat       0.69      0.82      0.75      1746\n\n    accuracy                           0.81      5000\n   macro avg       0.79      0.81      0.80      5000\nweighted avg       0.82      0.81      0.81      5000\n'

<a id="testing"></a>
### TESTING (evaluation.py)
<li><a href="#TableOfContents">Table of Contents</li></a>

In [23]:
# Accuracy
evaluation.accuracy(paws, 'actual')

actual: 100.00%

model1: 80.74%

model2: 63.04%

model3: 50.96%

model4: 74.26%

baseline: 65.08%



In [24]:
# Precision
evaluation.precision(paws, 'actual', 'dog', 'cat')

actual: 100.00%

model1: 80.33%

model2: 49.08%

model3: 50.86%

model4: 95.57%

baseline: 100.00%



In [25]:
# Recall
evaluation.recall(paws, 'actual', 'dog', 'cat')

actual: 100.00%

model1: 89.00%

model2: 89.32%

model3: 65.99%

model4: 73.12%

baseline: 65.08%



In [26]:
# Sensitivity (True Positive Rate)
evaluation.sensitivity_true_positive_rate(paws, 'actual', 'dog', 'cat')

actual: 100.00%

model1: 89.00%

model2: 89.32%

model3: 65.99%

model4: 73.12%

baseline: 65.08%



In [27]:
# Specificity
evaluation.specificity(paws, 'actual', 'dog', 'cat')

actual: 100.00%

model1: 68.98%

model2: 48.41%

model3: 35.83%

model4: 80.72%

baseline: nan%



/Users/manupfool/codeup-data-science/classification-exercises/evaluation.py:59: RuntimeWarning: invalid value encountered in long_scalars
  result = (matrix[1, 1] / (matrix[0, 1] + matrix[1, 1]))


In [28]:
# Negative Predicitive Value
evaluation.negative_predictive_value(paws, 'actual', 'dog', 'cat')

actual: 100.00%

model1: 81.50%

model2: 89.06%

model3: 51.15%

model4: 34.54%

baseline: 0.00%



In [29]:
# F1 Score
evaluation.f1_score(paws, 'actual', 'dog', 'cat')

actual: 50.00%

model1: 42.22%

model2: 31.67%

model3: 28.72%

model4: 41.43%

baseline: 39.42%

